In [3]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 13.3 MB/s eta 0:00:00


In [12]:
from google.colab import drive
import mediapipe as mp
import cv2
import json
import numpy as np
import os

drive.mount('/content/drive')

with open('/content/drive/MyDrive/4080/Data_convert/clip_5i9vRk-pAWY_99.pose_data.json', 'r') as file:
    pose_data = json.load(file)

output_images_folder = '/content/drive/MyDrive/4080/framesTest'
os.makedirs(output_images_folder, exist_ok=True)

part_colors = {
    'face': (0, 0, 255),
    'pose': (0, 255, 0),
    'left_hand': (255, 165, 0),
    'right_hand': (255, 0, 0)
}

def draw(image, landmarks, connections, part_colors):

    for start, end in connections:
        start_landmark = landmarks[start]
        end_landmark = landmarks[end]
        if start_landmark and end_landmark:

            cv2.line(image, start_landmark, end_landmark, part_colors, 2)


    for landmark in landmarks:
        cv2.circle(image, landmark, 3, part_colors, -1)

#https://stackoverflow.com/questions/73879333/how-to-only-draw-the-body-points-and-connection-without-the-face
#https://stackoverflow.com/questions/75365431/mediapipe-display-body-landmarks-only
bone_connections = {'face': [],'pose': [],'left_hand': [],'right_hand': []}
bone_connections['pose'] = [(11,12),(11,13),(13,15),(15,21),(15,19),(15,17),(17,19),(12,14),(14,16),(16,22),(16,20),(16,18),(18,20),(12,24),(11,23),(24,23),(24,26),(26,28),(28,32),(32,30),(28,30),(23,25),(25,27),(27,29),(27,31),(29,31)]
bone_connections['left_hand']= [(0,1), (1,2), (2,3), (3,4),(0,5), (5,6), (6,7), (7,8),(0,9), (9,10), (10,11), (11,12),(0,13), (13,14), (14,15), (15,16),(0,17), (17,18), (18,19), (19,20),(5,9),(9,13),(13,17)]
bone_connections['right_hand']= [(0,1), (1,2), (2,3), (3,4),(0,5), (5,6), (6,7), (7,8),(0,9), (9,10), (10,11), (11,12),(0,13), (13,14), (14,15), (15,16),(0,17), (17,18), (18,19), (19,20),(5,9),(9,13),(13,17)]


for frame_i, frame in enumerate(pose_data):
    image = np.ones((1080, 1920, 3), dtype=np.uint8) * 255


    for part, connections in bone_connections.items():
      part_landmarks = frame.get(part)
      #print("frame_i: ",frame_i)
      #print("frame: ",frame)
      #print("part_landmarks: ",part_landmarks)
      #print("connections: ",connections)

      if part_landmarks is None:
        part_landmarks = []
      landmarks = []
      for i in part_landmarks:
        if i:
          x = int(i['x'] * 1920)
          y = int(i['y'] * 1080)
          landmarks.append((x, y))
          #print("landmarks: ", landmarks)
        else:
          landmarks.append(None)

      if connections:
        max_i = 0

        for pair in connections:
          for i in pair:

            if i > max_i:
                max_i = i

        #print("max index: ", max_index)
        #print("miss: ", 1- len(landmarks))
        landmarks.extend([None]* (max_i+ 1- len(landmarks)))

      draw(image, landmarks, connections, part_colors[part])




    filename = f'frame_{frame_i:04d}.png'
    file_path = os.path.join(output_images_folder, filename)
    cv2.imwrite(file_path, image)


output_video_path = '/content/drive/MyDrive/4080/video_Test.mp4'
os.system(f"ffmpeg -framerate 30 -i {output_images_folder}/frame_%04d.png {output_video_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


256